In [1]:
print("hello world")

hello world


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

class BreastCancerMLP(nn.Module):
    def __init__(self, input_dim, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        return self.net(x)


In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

bc = datasets.load_wine()
df = pd.DataFrame(bc.data, columns=bc.feature_names)
targets = bc.target

# Ready data
X_train, X_test, y_train, y_test = train_test_split(df, targets, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # "fit" learns the mean/variance of each feature
X_test = scaler.transform(X_test)  # fit isn't necessary because the scaler already learned the features

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

print(df)

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5       95.0           1.68   
174    13.40        3.91  2.48               23.0      102.0           1.80   
175    13.27        4.28  2.26               20.0      120.0           1.59   
176    13.17        2.59  2.37               20.0      120.0           1.65   
177    14.13        4.10  2.74               24.5       96.0           2.05   

     flavanoids  nonflavanoid_phenols  proanthocyan

In [4]:
input_dim = df.shape[1]
num_classes = 3
model = BreastCancerMLP(input_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()     # backprop
    optimizer.step()    # update gradients

    print(f"Epoch {epoch+1}/{num_epochs}: loss {loss}")


# Verify that weights can be saved
path = "wine_mlp_weights.pth"
torch.save(model.state_dict(), path)
print("Model weights saved to " + path)

loaded_model = BreastCancerMLP(input_dim, num_classes)
loaded_model.load_state_dict(torch.load(path))
loaded_model.eval()
print("Model weights reloaded successfully.")

predictions = loaded_model(X_test)
_, predicted_classes = torch.max(predictions, 1)
accuracy = (predicted_classes == y_test).float().mean()
print(f"Test Accuracy: {accuracy.item()*100:.2f}%")
print(type(X_train))
print(X_train.shape)
print(X_test.shape)

Epoch 1/100: loss 1.100590467453003
Epoch 2/100: loss 1.092118501663208
Epoch 3/100: loss 1.0838168859481812
Epoch 4/100: loss 1.0755958557128906
Epoch 5/100: loss 1.0673668384552002
Epoch 6/100: loss 1.0590959787368774
Epoch 7/100: loss 1.050837516784668
Epoch 8/100: loss 1.0427186489105225
Epoch 9/100: loss 1.0347018241882324
Epoch 10/100: loss 1.0267447233200073
Epoch 11/100: loss 1.0189114809036255
Epoch 12/100: loss 1.0111777782440186
Epoch 13/100: loss 1.0035320520401
Epoch 14/100: loss 0.9958717226982117
Epoch 15/100: loss 0.98821622133255
Epoch 16/100: loss 0.9805316925048828
Epoch 17/100: loss 0.9727790951728821
Epoch 18/100: loss 0.9649282097816467
Epoch 19/100: loss 0.9569790363311768
Epoch 20/100: loss 0.9488924145698547
Epoch 21/100: loss 0.9406372904777527
Epoch 22/100: loss 0.9322656989097595
Epoch 23/100: loss 0.9237792491912842
Epoch 24/100: loss 0.9151788353919983
Epoch 25/100: loss 0.9064528942108154
Epoch 26/100: loss 0.8975515365600586
Epoch 27/100: loss 0.88847690